In [1]:
import os
import json
"""
sequences = []
def read_bytes(file_path, outputh_path):
    
    with open(file_path, 'rb') as f:
        byte_sequence = ["0x{:02x}".format(b) for b in f.read()]
    
    with open(output_path, 'w') as f:
      json.dump(byte_sequence, f)

output_directory = "./processed_data"
for root, dirs, files in os.walk(r"D:\json_specified_top10"):
  for name in files:
    #print(os.path.join(root, name))
    output_path = os.path.join(output_directory, name + ".json")
    read_bytes(os.path.join(root, name), outputh_path=output_path)
"""

'\nsequences = []\ndef read_bytes(file_path, outputh_path):\n    \n    with open(file_path, \'rb\') as f:\n        byte_sequence = ["0x{:02x}".format(b) for b in f.read()]\n    \n    with open(output_path, \'w\') as f:\n      json.dump(byte_sequence, f)\n\noutput_directory = "./processed_data"\nfor root, dirs, files in os.walk(r"D:\\json_specified_top10"):\n  for name in files:\n    #print(os.path.join(root, name))\n    output_path = os.path.join(output_directory, name + ".json")\n    read_bytes(os.path.join(root, name), outputh_path=output_path)\n'

In [2]:
import shutil

json_file_path = "top10label_data.json"
smaller_folder_path = "top10families"
large_folder_path = "processed_data"
"""
with open(json_file_path) as json_file:
    data = json.load(json_file)
    print(data.values())

for virus_type in set(data.values()):
    folder_path = os.path.join(smaller_folder_path, virus_type)
    os.makedirs(folder_path, exist_ok=True)

for filename, virus_type in data.items():
    source_image_path = os.path.join(large_folder_path, filename + '.json')
    destination_folder = os.path.join(smaller_folder_path, virus_type)
    destination_image_path = os.path.join(destination_folder, filename + '.json')

    # Copy the file to the destination folder
    if os.path.isfile(source_image_path):
        shutil.copyfile(source_image_path, destination_image_path)
        print(f"Copied image '{filename}.json' to '{virus_type}' folder.")
    else:
        print(f"Image file '{filename}.json' not found in the large folder, skipping")
"""
print("File grouping done")

File grouping done


In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

class_dirs =  [d for d in os.listdir(smaller_folder_path) if os.path.isdir(os.path.join(smaller_folder_path, d))]

file_paths = []
labels = []
x = []

for class_dir in class_dirs:
    class_folder = os.path.join(smaller_folder_path, class_dir)
    class_files = [os.path.join(class_folder, f) for f in os.listdir(class_folder) if f.endswith('.json')]
    file_paths.extend(class_files)
    labels.extend([class_dir]*len(class_files))

for file_path in file_paths:
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        json_subset = json_data[:60] # first 50
        feature = [int(x, 16) for x in json_data]
        x.append(feature)

y_data = np.array(labels)
x = np.array(x)
X_train, X_test, y_train, y_test = train_test_split(x, y_data, test_size=0.2, random_state=42)

C:\Users\turtl\AppData\Local\Temp\ipykernel_4636\2874012518.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
maxlen = 500
 
X_train = pad_sequences(X_train, maxlen=maxlen, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='post', truncating='post')
X_train_reshaped = X_train.reshape((*X_train.shape, 1))
X_test_reshaped = X_test.reshape((*X_test.shape, 1))

model = Sequential()
model.add(LSTM(128))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy')
#model.summary()
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

Epoch 1/10


ValueError: in user code:

    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Python310\lib\site-packages\keras\engine\input_spec.py", line 232, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer "sequential" "                 f"(type S